In [1]:
# Concurency with Futures.

In [2]:
import json
albums_id = json.load(open('albums_ids.json', 'r'))

In [3]:
albums_id

['1iSsdlURK7CGUVlcz4M5Li',
 '007KuGgJRyLlPvIkGtgqQV',
 '7maywFP1bQZyit0l9l0VHb',
 '5uEm5O3gMHJ1AZRQxSvDPV',
 '1M6ArhCSxqwEkKwei79qhK',
 '1y6lNpSBJck5Fi0iRwHot1',
 '50wYRXPk6GTJ6xDA3dQqas',
 '26jbVB8h8umHU1jEaUf8LZ',
 '2GadAFQi9OW9PrAB3519Fg',
 '1tVEWlyn2nsAke3Lp7HoZ0',
 '2FviQq6iUOb4FBrnySgWuh',
 '6wuR99S9OuyyWN5tD1EMQZ',
 '2EIvvMqgrCHORCpQBmJDjm',
 '7EIjgr1QML6dfSsShfwgE0',
 '63enh9Qs0Ec3AssU4wMTI1',
 '5Iugg1hYMiPy4KXEIPrShM',
 '33knvUBRR5ekNBrtLqQVTD',
 '1yssEiclrpPwxfzpMyHaHb',
 '24FEHm9O5fBnBtolT1jP41',
 '2AoBiklyaR5oKHfnN67fea']

In [4]:
import requests

In [5]:
CLIENT_ID = 'e88befb3447440e68cffa1fd5e8b395f'
CLIENT_SECRET = '3a99fd897bac420eab3149d75cf36624'

In [6]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [7]:
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

In [8]:
auth_response

<Response [200]>

In [9]:
auth_response_data = auth_response.json()

In [10]:
access_token = auth_response_data['access_token']

In [11]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [12]:
TRACKS_NAMES = []
BASE_URL = 'https://api.spotify.com/v1/'

In [13]:
def get_tracks(album_id):
    r = requests.get(BASE_URL + 'albums/' + album_id + '/tracks', 
                headers=headers)
    print(r.json())
    data = r.json()['items']
    return [track['name'] for track in data]

In [14]:
from concurrent import futures

In [15]:
MAX_WORKERS = 20
workers = min(MAX_WORKERS, len(albums_id))

In [16]:
with futures.ThreadPoolExecutor(workers) as executor:
    to_do = []
    for album_id in sorted(albums_id):
        future = executor.submit(get_tracks, album_id)
        to_do.append(future)
    results = []
    for future in futures.as_completed(to_do):
        res = future.result()
        TRACKS_NAMES.extend(res)

{'href': 'https://api.spotify.com/v1/albums/24FEHm9O5fBnBtolT1jP41/tracks?offset=0&limit=20', 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2NPduAUeLVsfIauhRwuft1'}, 'href': 'https://api.spotify.com/v1/artists/2NPduAUeLVsfIauhRwuft1', 'id': '2NPduAUeLVsfIauhRwuft1', 'name': 'Nightwish', 'type': 'artist', 'uri': 'spotify:artist:2NPduAUeLVsfIauhRwuft1'}], 'available_markets': ['AU', 'CA', 'JP', 'NZ', 'US'], 'disc_number': 1, 'duration_ms': 155480, 'explicit': False, 'external_urls': {'spotify': 'https://open.spotify.com/track/1HPR2tBIsEVmMce1k481R6'}, 'href': 'https://api.spotify.com/v1/tracks/1HPR2tBIsEVmMce1k481R6', 'id': '1HPR2tBIsEVmMce1k481R6', 'is_local': False, 'name': 'Taikatalvi', 'preview_url': 'https://p.scdn.co/mp3-preview/7c7b5ec8193ab8b7f16fd1aa02eec989555628d7?cid=e88befb3447440e68cffa1fd5e8b395f', 'track_number': 1, 'type': 'track', 'uri': 'spotify:track:1HPR2tBIsEVmMce1k481R6'}, {'artists': [{'external_urls': {'spotify': 'https://o

In [17]:
TRACKS_NAMES

['Taikatalvi',
 'Storytime',
 'Ghost River',
 'Slow, Love, Slow',
 'I Want My Tears Back',
 'Scaretale',
 'Arabesque',
 'Turn Loose the Mermaids',
 'Rest Calm',
 'The Crow, the Owl and the Dove',
 'Last Ride of the Day',
 'Song of Myself',
 'Imaginaerum',
 'Shudder Before the Beautiful',
 'Weak Fantasy',
 'Élan',
 'Yours Is an Empty Hope',
 'Our Decades in the Sun',
 'My Walden',
 'Endless Forms Most Beautiful',
 'Edema Ruh',
 'Alpenglow',
 'The Eyes of Sharbat Gula',
 'The Greatest Show on Earth',
 'Shudder Before the Beautiful - Instrumental',
 'Weak Fantasy - Instrumental',
 'Élan - Instrumental',
 'Yours Is an Empty Hope - Instrumental',
 'Our Decades in the Sun - Instrumental',
 'My Walden - Instrumental',
 'Endless Forms Most Beautiful - Instrumental',
 'Edema Ruh - Instrumental',
 'Alpenglow - Instrumental',
 'Taikatalvi',
 'Storytime',
 'Ghost River',
 'Slow, Love, Slow',
 'I Want My Tears Back',
 'Scaretale',
 'Arabesque',
 'Turn Loose the Mermaids',
 'Rest Calm',
 'The Crow, 

In [18]:
TRACKS_NAMES = []

In [19]:
%%timeit -n 1 -r 1
with futures.ThreadPoolExecutor(workers) as executor:
    to_do = []
    for album_id in sorted(albums_id):
        future = executor.submit(get_tracks, album_id)
        to_do.append(future)
    results = []
    for future in futures.as_completed(to_do):
        res = future.result()
        TRACKS_NAMES.extend(res)

{'href': 'https://api.spotify.com/v1/albums/1iSsdlURK7CGUVlcz4M5Li/tracks?offset=0&limit=20', 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2NPduAUeLVsfIauhRwuft1'}, 'href': 'https://api.spotify.com/v1/artists/2NPduAUeLVsfIauhRwuft1', 'id': '2NPduAUeLVsfIauhRwuft1', 'name': 'Nightwish', 'type': 'artist', 'uri': 'spotify:artist:2NPduAUeLVsfIauhRwuft1'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 